<a href="https://colab.research.google.com/github/adas754/ai-agent/blob/main/langchain_using_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q langchain_google_genai

!pip install -q langchain


!pip install -q langchain_community

In [3]:
import os


os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "lsv2_pt_a078c079aaf1453bac0a98425daa5378_1ef4e06f7dqqqqqqqqqqqqqqqqqqqqqqqqq"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "project"
os.environ["GOOGLE_API_KEY"] = "AIzaSyDTPDqOhCBxkw_YN3hWXqgPIjtZkPFvQyMqqqqqqqqqqqqqqqqqqqqqq"

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI


model = ChatGoogleGenerativeAI(model="gemini-pro",convert_system_message_to_human=True)



model.invoke("hi").content


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:353: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Hello there! How can I assist you today?'

In [6]:
from langchain_core.output_parsers import StrOutputParser


parser = StrOutputParser()


parser.invoke(model.invoke("hi"))


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:353: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Hello! How can I help you today?'

In [7]:
from langchain_core.messages import HumanMessage


import warnings
warnings.filterwarnings("ignore")


while True:
  message = input("Write your query:")
  if message=="bye":
    print("Good Bye have a great day!")
    break
  else:
    print(parser.invoke(model.invoke([HumanMessage(content=message)])))


Write your query:hi
Hello there! How can I assist you today?
Write your query:bye
Good Bye have a great day!


In [11]:
from langchain_core.chat_history import BaseChatMessageHistory


from langchain_core.chat_history import InMemoryChatMessageHistory


from langchain_core.runnables.history import RunnableWithMessageHistory


from langchain_core.messages import AIMessage


result = model.invoke(
    [
        HumanMessage(content="Hi! I'm cristinao"),
        AIMessage(content="Hello cristinao! How can I assist you today?"),
        HumanMessage(content="What's my name?")
    ]
)


parser.invoke(result)


'Your name is cristinao, as you have told me. Is there anything else I can help you with today?'

In [14]:
store={}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


config = {"configurable": {"session_id": "firstchat"}}


model_with_memory=RunnableWithMessageHistory(model,get_session_history)


model_with_memory.invoke([HumanMessage(content="Hi! I'm munai")],config=config).content

'Hi munai, I am Gemini, a multi-modal AI language model developed by Google. I am designed to provide information and assist people with a wide range of language-related tasks. Is there anything I can help you with today?'

In [15]:
model_with_memory.invoke([HumanMessage(content="what is my name?")],config=config).content


'Your name is munai. You told me so yourself in your first message to me.'

In [16]:
config = {"configurable": {"session_id": "secondtchat"}}


config


{'configurable': {'session_id': 'secondtchat'}}

In [17]:
model_with_memory.invoke([HumanMessage(content="what is my name?")],config=config).content

'I do not have access to your personal information, so I cannot provide you with your name.'

In [18]:
config = {"configurable": {"session_id": "firstchat"}}


config


{'configurable': {'session_id': 'firstchat'}}

In [19]:
store


{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm munai", additional_kwargs={}, response_metadata={}), AIMessage(content='Hi munai, I am Gemini, a multi-modal AI language model developed by Google. I am designed to provide information and assist people with a wide range of language-related tasks. Is there anything I can help you with today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-32460d73-0cb4-41dc-af2e-416a51668d41-0', usage_metadata={'input_tokens': 8, 'output_tokens': 48,

In [20]:
model_with_memory.invoke([HumanMessage(content="what is my name?")],config=config).content

'You have already told me that your name is munai. If this is not correct, please let me know what your name is and I will update my records.'

In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


prompt=ChatPromptTemplate.from_messages(
    [
      ("system","You are a helpful assistant. Answer all questions to the best of your ability.",),
      MessagesPlaceholder(variable_name="messages"),
    ]
)


chain = prompt | model


chain.invoke({"messages": ["hi! I'm bob"]})


AIMessage(content='Hi Bob! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-5e969963-3ed0-4510-a133-23abecceac6f-0', usage_metadata={'input_tokens': 23, 'output_tokens': 10, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}})

In [22]:
chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]}).content

'Hi Bob! How can I help you today?'

In [23]:

chain.invoke({"messages": [HumanMessage(content="what is my name")]}).content

'I do not have access to your personal information, so I cannot tell you your name.'

In [33]:
model_with_memory=RunnableWithMessageHistory(chain,get_session_history)


config = {"configurable": {"session_id": "thirdchat"}}


response=model_with_memory.invoke(
    [HumanMessage(content="Hi! I'm Jim"),
     ],config=config
)


response.content

'Nice to meet you, Jim! How can I assist you today?'

In [34]:

response=model_with_memory.invoke(
    [HumanMessage(content="hi what is my name"),
     ],config=config
)
print(response.content)

You are Jim, as you mentioned in your previous message. Is there anything else I can help you with today?


In [26]:
response=model_with_memory.invoke(
    [HumanMessage(content="what is 2+2?"),
     ],config=config
)
print(response.content)

2 + 2 is 4.


In [27]:

response=model_with_memory.invoke(
    [HumanMessage(content="who is a indian cricket team caption?"),
     ],config=config
)
print(response.content)


The current captain of the Indian cricket team is Rohit Sharma.


In [29]:
response=model_with_memory.invoke(
    [HumanMessage(content="what should i add in previous addition so that i will become 10?"),
     ],config=config
)
print(response.content)


You need to add 4 to the previous addition (2 + 2) to get 10.
(2 + 2) + 4 = 10


In [35]:

response=model_with_memory.invoke(
    [HumanMessage(content="what is my name?"),
     ],config=config
)
print(response.content)

You are Jim, as you mentioned in your previous message. Is there anything else I can help you with today?


In [36]:

from langchain_core.messages import SystemMessage, trim_messages


trimmer = trim_messages(
    max_tokens=60,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)


messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [37]:
model.get_num_tokens_from_messages(messages)

51

In [38]:
trimmer.invoke(messages)


[HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [41]:
trimmed_message = trimmer.invoke(messages)


model.get_num_tokens_from_messages(trimmed_message)


51

In [40]:
prompt=ChatPromptTemplate.from_messages(
    [
      ("system","You are a helpful assistant. Answer all questions to the best of your ability.",),
      MessagesPlaceholder(variable_name="messages"),
    ]
)


messages


[HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [44]:

from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I don't know your name."

In [43]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content


'2 + 2'

In [45]:
model_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)



config = {"configurable": {"session_id": "fifthchat"}}


response = model_with_memory.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content


'I do not know your name.'

In [46]:
response = model_with_memory.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'2 + 2'